# 05 - Transactions & Bulk Operations

## Use Case: E-Commerce Checkout

In an e-commerce system, a checkout must be **atomic**: create the order, record the payment,
and decrement inventory stock — all or nothing. If the payment fails, the order should not
exist and inventory should remain unchanged.

This notebook covers:

1. **Transactions** — Atomic multi-operation blocks with automatic rollback
2. **Bulk Create** — Insert many records in a single round-trip
3. **Bulk Update** — Update filtered records in batch
4. **Bulk Delete** — Remove filtered records in batch
5. **Atomic Array Operations** — Server-side array mutations without read-modify-write
6. **Retry on Conflict** — Automatic retry for concurrent write conflicts

## Prerequisites

- SurrealDB running locally (`docker run --rm -p 8000:8000 surrealdb/surrealdb:latest start --user root --pass root`)
- Python packages installed (`uv sync` in the project root)

In [ ]:
# Setup: add project root to path and configure the connection
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)

## Model Definitions

We define three models that represent a simplified e-commerce domain:

- **Product** — Items in the catalog with a stock count
- **Order** — A customer's purchase with status tracking
- **Payment** — The financial transaction linked to an order

In [ ]:
# Define our e-commerce models
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict
from datetime import datetime


class Product(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="products")

    id: str | None = None
    name: str
    price: float
    stock: int = 0
    tags: list[str] = []


class Order(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="orders")

    id: str | None = None
    customer_name: str
    product_id: str
    quantity: int
    total: float
    status: str = "pending"  # pending, processing, completed, cancelled


class Payment(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="payments")

    id: str | None = None
    order_id: str
    amount: float
    method: str = "credit_card"
    status: str = "confirmed"


print("Models defined: Product, Order, Payment")

In [ ]:
# Create some sample products to work with
laptop = Product(name="Laptop Pro 16", price=1499.99, stock=25, tags=["electronics", "computers"])
await laptop.save()

headphones = Product(name="Wireless Headphones", price=79.99, stock=100, tags=["electronics", "audio"])
await headphones.save()

keyboard = Product(name="Mechanical Keyboard", price=129.99, stock=50, tags=["electronics", "peripherals"])
await keyboard.save()

print(f"Created products: {laptop.name} (stock: {laptop.stock}), "
      f"{headphones.name} (stock: {headphones.stock}), "
      f"{keyboard.name} (stock: {keyboard.stock})")

## 1. Transactions — Atomic Checkout

A transaction groups multiple database operations into a single atomic unit.
If **any** operation fails, **all** operations are rolled back.

This is essential for e-commerce: you never want an order to exist without a matching
payment, or inventory to be decremented without a confirmed order.

```
async with SurrealDBConnectionManager.transaction() as tx:
    await model.save(tx=tx)
    # If anything raises, everything is rolled back
```

In [ ]:
# Successful atomic checkout: order + payment + stock decrement
async with SurrealDBConnectionManager.transaction() as tx:
    # Step 1: Create the order
    order = Order(
        customer_name="Alice",
        product_id=f"products:{laptop.id}",
        quantity=1,
        total=laptop.price,
        status="confirmed",
    )
    await order.save(tx=tx)

    # Step 2: Create the payment
    payment = Payment(
        order_id=f"orders:{order.id}",
        amount=laptop.price,
        method="credit_card",
    )
    await payment.save(tx=tx)

    # Step 3: Decrement stock
    await laptop.merge(stock=laptop.stock - 1, tx=tx)

# Transaction committed — all three operations succeeded
print(f"Order created: {order.id} (status: {order.status})")
print(f"Payment created: {payment.id} (amount: ${payment.amount})")

# Verify stock was decremented
refreshed_laptop = await Product.objects().get(laptop.id)
print(f"Laptop stock: {refreshed_laptop.stock} (was 25)")

In [ ]:
# Demonstrating rollback: simulate a failed checkout
# We record stock before to show it remains unchanged after rollback
stock_before = (await Product.objects().get(headphones.id)).stock
print(f"Headphones stock before: {stock_before}")

try:
    async with SurrealDBConnectionManager.transaction() as tx:
        # Create order (would succeed)
        bad_order = Order(
            customer_name="Bob",
            product_id=f"products:{headphones.id}",
            quantity=1,
            total=headphones.price,
        )
        await bad_order.save(tx=tx)

        # Simulate payment failure (e.g., card declined)
        raise ValueError("Payment declined: insufficient funds")

        # This line is never reached
        await headphones.merge(stock=headphones.stock - 1, tx=tx)

except ValueError as e:
    print(f"Transaction rolled back: {e}")

# Verify stock is unchanged
stock_after = (await Product.objects().get(headphones.id)).stock
print(f"Headphones stock after rollback: {stock_after} (unchanged: {stock_before == stock_after})")

## 2. Bulk Create — Import Products in Batch

When inserting many records, `bulk_create()` is far more efficient than saving
one-by-one. With `atomic=True`, all records are inserted in a single transaction —
if one fails, none are created.

Use cases:
- Importing product catalogs from CSV/API
- Seeding test data
- Batch ETL pipelines

In [ ]:
# Bulk create 50 products in a single atomic operation
import random

categories = ["electronics", "clothing", "books", "home", "sports"]
adjectives = ["Premium", "Basic", "Pro", "Ultra", "Eco"]
nouns = ["Widget", "Gadget", "Tool", "Device", "Kit"]

bulk_products = [
    Product(
        name=f"{random.choice(adjectives)} {random.choice(nouns)} {i}",
        price=round(random.uniform(9.99, 299.99), 2),
        stock=random.randint(0, 200),
        tags=random.sample(categories, k=random.randint(1, 3)),
    )
    for i in range(50)
]

# atomic=True means all-or-nothing: if one fails, none are created
created = await Product.objects().bulk_create(bulk_products, atomic=True)
print(f"Bulk created {len(created)} products")
print(f"Sample: {created[0].name} — ${created[0].price} (stock: {created[0].stock})")

## 3. Bulk Update — Batch Status Changes

Use `bulk_update()` on a filtered `QuerySet` to update many records at once.
This is much faster than loading each record, modifying it, and saving.

Use cases:
- Moving all "pending" orders to "processing" when a batch processor runs
- Applying a price increase to all products in a category
- Deactivating all expired subscriptions

In [ ]:
# First, create some orders in various statuses
order_data = [
    Order(customer_name="Charlie", product_id="products:1", quantity=2, total=59.98, status="pending"),
    Order(customer_name="Diana", product_id="products:2", quantity=1, total=129.99, status="pending"),
    Order(customer_name="Eve", product_id="products:3", quantity=3, total=239.97, status="pending"),
    Order(customer_name="Frank", product_id="products:4", quantity=1, total=49.99, status="cancelled"),
    Order(customer_name="Grace", product_id="products:5", quantity=1, total=89.99, status="cancelled"),
]
await Order.objects().bulk_create(order_data)

# Count pending orders before
pending_before = await Order.objects().filter(status="pending").count()
print(f"Pending orders before: {pending_before}")

# Bulk update: move all pending orders to processing
await Order.objects().filter(status="pending").bulk_update({"status": "processing"})

# Verify the change
pending_after = await Order.objects().filter(status="pending").count()
processing = await Order.objects().filter(status="processing").count()
print(f"Pending orders after: {pending_after}")
print(f"Processing orders: {processing}")

## 4. Bulk Delete — Remove Cancelled Orders

Use `bulk_delete()` on a filtered `QuerySet` to delete many records at once.

Use cases:
- Purging cancelled or expired records
- Data retention cleanup (delete records older than N days)
- Test data teardown

In [ ]:
# Count cancelled orders before deletion
cancelled_count = await Order.objects().filter(status="cancelled").count()
print(f"Cancelled orders before: {cancelled_count}")

# Bulk delete all cancelled orders
await Order.objects().filter(status="cancelled").bulk_delete()

# Verify they are gone
cancelled_after = await Order.objects().filter(status="cancelled").count()
print(f"Cancelled orders after: {cancelled_after}")

## 5. Atomic Array Operations — Tagging System

In concurrent environments (e.g., multiple API pods), the classic read-modify-write
pattern for array fields causes race conditions:

```python
# DANGEROUS: Two pods read stock=["electronics"], both append, one write is lost
product = await Product.objects().get(product_id)
product.tags.append("sale")
await product.save()
```

Atomic operations solve this by executing the mutation server-side:

- **`atomic_append()`** — Append a value (allows duplicates)
- **`atomic_set_add()`** — Add only if not present (no duplicates)
- **`atomic_remove()`** — Remove a value from the array

In [ ]:
# Start with a fresh product to demonstrate atomic array operations
product = await Product.objects().get(keyboard.id)
print(f"Tags before: {product.tags}")

# atomic_set_add: add "sale" tag only if not already present
await Product.atomic_set_add(keyboard.id, "tags", "sale")
product = await Product.objects().get(keyboard.id)
print(f"After atomic_set_add('sale'): {product.tags}")

# atomic_set_add again: "sale" won't be duplicated
await Product.atomic_set_add(keyboard.id, "tags", "sale")
product = await Product.objects().get(keyboard.id)
print(f"After second atomic_set_add('sale'): {product.tags}")

# atomic_append: always appends (allows duplicates)
await Product.atomic_append(keyboard.id, "tags", "featured")
product = await Product.objects().get(keyboard.id)
print(f"After atomic_append('featured'): {product.tags}")

# atomic_remove: remove the "sale" tag
await Product.atomic_remove(keyboard.id, "tags", "sale")
product = await Product.objects().get(keyboard.id)
print(f"After atomic_remove('sale'): {product.tags}")

## 6. Retry on Conflict

When multiple processes write to the same record concurrently, SurrealDB may
raise a transaction conflict error. The `@retry_on_conflict` decorator
automatically retries the operation with exponential backoff and jitter.

This is especially useful in:
- Multi-pod Kubernetes deployments
- Background task workers processing the same queue
- Real-time applications with concurrent updates

```python
@retry_on_conflict(max_retries=3, base_delay=0.05)
async def process(record_id, value):
    await Model.atomic_set_add(record_id, "field", value)
```

In [ ]:
# Demonstrate retry_on_conflict decorator pattern
from src.surreal_orm import retry_on_conflict


@retry_on_conflict(max_retries=5, base_delay=0.05)
async def add_tag_safely(product_id: str, tag: str):
    """Add a tag to a product, retrying automatically if there's a write conflict."""
    await Product.atomic_set_add(product_id, "tags", tag)
    print(f"Successfully added tag '{tag}' to product {product_id}")


# In a real scenario, multiple pods would call this concurrently
# The decorator ensures eventual consistency
await add_tag_safely(keyboard.id, "best-seller")

# Verify the tag was added
product = await Product.objects().get(keyboard.id)
print(f"Final tags: {product.tags}")

## Cleanup

Remove all data created during this notebook.

In [ ]:
# Clean up all tables used in this notebook
await Product.raw_query("DELETE FROM products")
await Order.raw_query("DELETE FROM orders")
await Payment.raw_query("DELETE FROM payments")

print("Cleanup complete: all products, orders, and payments deleted.")

## Summary

| Feature | When to Use |
|---------|------------|
| **Transactions** | Multi-step operations that must all succeed or all fail |
| **Bulk Create** | Inserting many records at once (imports, seeding) |
| **Bulk Update** | Updating many records matching a filter |
| **Bulk Delete** | Deleting many records matching a filter |
| **Atomic Array Ops** | Concurrent array mutations without read-modify-write |
| **Retry on Conflict** | Wrapping operations that may hit concurrent write conflicts |